In [35]:
from torch.nn import functional as F
import torch
from torch import nn

In [36]:
class RNN(nn.Module):
    
    def __init__(self,input_size,hidden_size,output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size,hidden_size,batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)
    
    
    def forward(self,x,target):
        # B,T,C
        out,h = self.rnn(x)
        # B,T,H
        out = self.fc(out)
        loss = F.binary_cross_entropy(out,target)
        return loss

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Initialize the hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        # Pass the input through the RNN layer
        out, hn = self.rnn(x, h0)

        # Pass the output through the fully connected layer and the sigmoid activation function
        out = self.fc(out)
        
        out = F.softmax(out,dim=-1)

        return out.squeeze()

# Generate some random binary classification data
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

# Set up the RNN model
input_size = 10
hidden_size = 16
output_size = 1
rnn = RNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.AdamW(rnn.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    X = X_train.unsqueeze(0)
    Y = y_train.unsqueeze(0)
    # Forward pass
    outputs = rnn(X)

    # Compute the loss and backward pass
    Y = Y.squeeze(0)
    loss = criterion(outputs, Y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print the loss every 10 epochs
    if epoch % 10 == 0:
        print('Epoch %d Loss: %.4f' % (epoch, loss.item()))

# Test the model
with torch.no_grad():
    X = X_test.unsqueeze(0)
    y_pred = rnn(X)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0

accuracy = (y_pred == y_test).sum().item() / y_test.size(0)
print('Test Accuracy: %.4f' % accuracy)

Epoch 0 Loss: 49.6250
Epoch 10 Loss: 49.6250
Epoch 20 Loss: 49.6250
Epoch 30 Loss: 49.6250
Epoch 40 Loss: 49.6250
Epoch 50 Loss: 49.6250
Epoch 60 Loss: 49.6250
Epoch 70 Loss: 49.6250
Epoch 80 Loss: 49.6250
Epoch 90 Loss: 49.6250
Test Accuracy: 0.4800


In [ ]:
class DKT(nn.Module):

    def __init__(self,input_size,quesion_size):
        super().__init__()
        self.net = nn.Linear(input_size,quesion_size)
        self.sigmoid = nn.Sigmoid()
    
    
    def forward(self,x,target=None):
        # x B,T,C
        logits = self.net(x) # B,T,question_size
        # target B,T
        # B,T,C 
        B,T,C = logits.shape
        if target is not None:
            logs = logits.view(B*T,-1)
            target = target.view(B*T)
            loss =  F.cross_entropy(logs,target)
        else:
            loss = None
        return loss,logits

In [ ]:
import json

In [ ]:
num_question = 123

In [ ]:
import numpy as np

In [ ]:
block_size = 50

In [ ]:
def get_data(dtype):
    with open('{}.json'.format(dtype),'r') as f:
        train_data = json.load(f)
        result = []
        y_arr = []
        for x,y in train_data:
            x_len = len(x)
            p = block_size - x_len % block_size
            total = x_len + p
            one_hot = [[0]*num_question*2 for _ in range(total)]
            yy = y + [0]*p
            for i,(xi,yi) in enumerate(zip(x,y)):
                if yi == 1:
                    one_hot[i][xi] = 1.
                else:
                    one_hot[i][xi+num_question] = 1.
            result.extend(one_hot)
            y_arr.extend(yy)
        result = torch.tensor(result)
        print(result.shape)
        result = result.reshape(-1,block_size,num_question*2)
        y_arr = torch.tensor(y_arr)
        y_arr = y_arr.reshape(-1,block_size)
        return result,y_arr

In [ ]:
d = torch.tensor([[0,1,2,0]])

In [ ]:
def parse_raw_data(x):
    #x B,T,C
    



In [ ]:
x,y = get_data('train')

In [ ]:
x.dtype

In [ ]:
y.shape

In [ ]:
dkt = DKT(num_question*2,num_question)

In [ ]:
import torch.optim as optim
optimizer = optim.AdamW(dkt.parameters(),lr=0.0003)

In [ ]:
for xb,yb in zip(x,y):
    loss,logits = dkt(xb.unsqueeze(0),yb.unsqueeze(0))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

In [ ]:
val_x,val_y = get_data('val')

In [ ]:
loss,log=dkt(val_x,val_y)

In [ ]:
log.shape

In [ ]:
(torch.argmax(log,dim=-1) == val_y).sum() / val_y.numel()

In [ ]:
torch.argmax(log,dim=-1)

In [ ]:
log

In [ ]:
val_y

In [147]:
(torch.argmax(log,dim=-1) == val_y).sum() / val_y.numel()

tensor(0.9038)

In [145]:
torch.argmax(log,dim=-1)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 0,  ..., 0, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])

In [149]:
log

tensor([[[ 1.6134,  2.9911, -4.8053,  ..., -4.8083, -4.8109, -4.7659],
         [ 1.4352,  3.1362, -4.9730,  ..., -5.0383, -4.8650, -4.9196],
         [ 1.4721,  3.1411, -4.8628,  ..., -4.9298, -4.8128, -4.8126],
         ...,
         [ 3.1372,  1.8240, -3.7837,  ..., -3.7848, -3.7221, -3.7232],
         [ 3.1372,  1.8240, -3.7837,  ..., -3.7848, -3.7221, -3.7232],
         [ 3.1372,  1.8240, -3.7837,  ..., -3.7848, -3.7221, -3.7232]],

        [[ 1.6134,  2.9911, -4.8053,  ..., -4.8083, -4.8109, -4.7659],
         [ 1.4352,  3.1362, -4.9730,  ..., -5.0383, -4.8650, -4.9196],
         [ 4.1010,  0.5850, -4.7277,  ..., -4.7815, -4.6155, -4.6501],
         ...,
         [ 3.1372,  1.8240, -3.7837,  ..., -3.7848, -3.7221, -3.7232],
         [ 3.1372,  1.8240, -3.7837,  ..., -3.7848, -3.7221, -3.7232],
         [ 3.1372,  1.8240, -3.7837,  ..., -3.7848, -3.7221, -3.7232]],

        [[ 1.6134,  2.9911, -4.8053,  ..., -4.8083, -4.8109, -4.7659],
         [ 1.4352,  3.1362, -4.9730,  ..., -5

In [146]:
val_y

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 0,  ..., 0, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])